# Parsing Randomness
This is an example showing the principles of "free generators" for type `a`, a data structure interpretable as:
- A generator for type `a`,
- A parser taking a string and creating a type `a`, and 
- A generator creating an input string for the parser

First, like in the paper [Parsing Randomness](https://dl.acm.org/doi/10.1145/3563291), the "original" parser and generator will be shown. Next, they will both be build using the introduced data structure `FreeGen`. Finally, the separation of a generator into a parser and a source of randomness is shown.

The utilised example are trees, first introduced.

## Tree Example
The utilized trees will have the following elements:
- `N` stands for a node
  - A node has 2 children (other tree), and
  - An assigned digit (`0..9`)
- `L` stands for leaf

An example could look like this:
```haskell
N: 2 
|- L
`- N: 1
   |- N: 6 -> L L
   `- L
```

The traditional generator and parser have the following tasks:
- A **generator** creates such a data structure at random. 
- A **parser** gets some string input (for the shown example: `N 2 L N 1 N 6 L L L`) and creates the according instance of the data structure.


In [128]:
:reload
:load ./Definitions/Tree.hs

import Definitions.Tree

## Implementation of original Generator and Parser
To show the analogy of the structure in generators and parsers, this section will start with the original implementation of both for creating (either making or parsing) a tree.

Both the utilised generator and parser are already existing implementations from other libraries.

In [129]:
import Test.QuickCheck (Gen)
import qualified Test.QuickCheck.Gen as Gen

generateTree :: Int -> Gen Tree
generateTree 0 = do
  c <- Gen.frequency [(1, return 'L')]
  case c of
    'L' -> return Leaf
generateTree h = do
  c <- Gen.frequency [(1, return 'L'), (3, return 'N')]
  case c of
    'L' -> return Leaf
    'N' -> do
      x <- generateInt
      l <- generateTree (h-1)
      r <- generateTree (h-1)
      return (Node x l r)

generateInt = Gen.frequency [ (1, return x) | x<-[0..9] ]

Gen.generate (generateTree 4)


N: 3
|- L
`- N: 8
   |- N: 5
   |  |- N: 9 -> L L
   |  `- N: 3 -> L L
   `- N: 7
      |- N: 0 -> L L
      `- N: 9 -> L L

In [130]:
import Text.Parsec.String (Parser)
import Text.Parsec
import Data.Char

parseTree :: Int -> Parser Tree
parseTree 0 = do
  c <- oneOf "L"
  case c of 
    'L' -> return Leaf
parseTree h = do
  c <- oneOf "LN"
  case c of
    'L' -> return Leaf
    'N' -> do
      x <- parseInt
      l <- parseTree (h-1)
      r <- parseTree (h-1)
      return (Node x l r)

parseInt = digitToInt <$> digit

parse (parseTree 5) "" "N2LN1N6LLL"

Right 
N: 2
|- L
`- N: 1
   |- N: 6 -> L L
   `- L

## Implementation of Free Generator
As a first step, the general free generator for the tree exampale will be defined. 

Important to notice: the resulting output `FreeGen Tree` is simply an abstract data structure. Without the according interpretation, it is neither a generator nor a parser. Still, the structure has a remarkable resemblence with both the definition to get a `Gen Tree` and a `Parser Tree`.

In [131]:
:reload
:load ./Definitions/FreeGen.hs

import Definitions.FreeGen

In [132]:
freeGenTree :: Int -> FreeGen Tree
freeGenTree 0 = do -- Basically says: always return Leaf, BUT captures this in the distribution
  c <- pick [(1, 'L', return 'L')]
  case c of
    'L' -> return Leaf
freeGenTree h = do
  c <- pick [(1, 'L', return 'L'), (3, 'N', return 'N')]
  case c of
    'L' -> return Leaf
    'N' -> do
      x <- freeGenInt
      l <- freeGenTree (h-1)
      r <- freeGenTree (h-1)
      return (Node x l r)

freeGenInt = pick [(1, intToDigit x, return x) | x<-[0..9]]

freeGen = freeGenTree 4

## Interpretations for FreeGen

Now follows the interesting part about the created `FreeGen Tree`: the different interpretations. The results should be
- a Generator (`Gen Tree`)
- a Parser (`Parser Tree`)

In [133]:
gen = interpretAsG freeGen
Gen.generate gen


N: 8
|- N: 1
|  |- L
|  `- N: 5
|     |- N: 5 -> L L
|     `- L
`- N: 2
   |- L
   `- N: 7 -> L L

In [134]:
parser = interpretAsP freeGen
parse parser "" "N2LN1N6LLL"

Right 
N: 2
|- L
`- N: 1
   |- N: 6 -> L L
   `- L

From the two upper code segments it can be shown, that both the interpretation as a generetor and parser work. 

To show that a generator can now be indeed separated into a parser and a generator for random sequences, an additional interpretation needs to be used:
- Randomness interpretation (`Gen String`)

In [135]:
random = interpretAsR freeGen

Gen.generate random

"N1N6N9N9LLN5LLLN8N0N5LLLN8N1LLN1LL"


With the resulting randomness generator (that creates random strings), the generator for type `Tree` can then be factored into the parser and the distribution over choice sequences.

In [136]:
Gen.generate (parse parser "" <$> random)

Right 
N: 9
|- N: 6
|  |- N: 5
|  |  |- N: 7 -> L L
|  |  `- N: 6 -> L L
|  `- N: 8
|     |- N: 9 -> L L
|     `- N: 2 -> L L
`- N: 0
   |- N: 7
   |  |- N: 2 -> L L
   |  `- N: 1 -> L L
   `- N: 3
      |- N: 5 -> L L
      `- N: 2 -> L L